<a href="https://colab.research.google.com/github/travelmail26/projectsleep/blob/main/Muse_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne
!pip install yasa
!pip install datashader
import pandas as pd
import mne
import numpy as np
import yasa
import matplotlib as plt
import datashader as ds
import datashader.transfer_functions as tf
from IPython.display import display
import seaborn as sns



In [ ]:
##load files

#dataFile = pd.read_csv("/content/stimulant_mindMonitor_2023-01-18--14-37-14.csv", skiprows=0, usecols=[*range(1,21)])
#url = 'https://www.dropbox.com/s/ttv6vxh3ys7q0ou/mindMonitor_2023-01-18--14-37-14.csv?dl=1'
url = 'https://www.dropbox.com/s/a5uwkqaq71mkrxp/mindMonitor_2023-02-17--18-29-44_choc_intervention.csv?dl=1'
urlmantrameditation = 'https://www.dropbox.com/s/z09umrre7qgg4ld/mindMonitor_2023-02-23--08-01-50_eyesopen_eyesclose_mantrameditation_eyesopen_eyesclosed.csv?dl=1'
urlmantra2 = 'https://www.dropbox.com/s/z0eoyhwinrep224/mindMonitor_2023-03-28--10-43-44_after_breakfast_baseline_mantra__3_intervention_breathfocus_4.csv?dl=1'
#dataFile = pd.read_csv('/content/mindMonitor_2023-01-18--14-37-14.csv', skiprows=0, usecols=[*range(1,21)])
urlsleep = 'https://www.dropbox.com/s/hu50afgorep21c6/mindMonitor_2023-04-03--01-28-07_sleep.csv?dl=1'
#dfraw = pd.read_csv('/content/mindMonitor_2023-01-18--14-37-14.csv')

#dfraw = pd.read_csv('/content/mindMonitor_2023-02-15--14-48-03_stiumant_limited_interventions.csv')
#dfraw = pd.read_csv('/content/mindMonitor_2023-02-23--08-01-50_eyesopen_eyesclose_mantrameditation_eyesopen_eyesclosed.csv')
#dfraw = pd.read_csv('/content/mindMonitor_2023-02-23--08-01-50_eyesopen_eyesclose_mantrameditation_eyesopen_eyesclosed.csv')
#urlsleep58 = 'https://www.dropbox.com/s/9r516639sms98sj/mindMonitor_2023-05-07--23-31-37_5188390995226691787_sleep.csv?dl=1'
urlsleep614 = 'https://www.dropbox.com/s/s21a5x5z1riy8jt/mindMonitor_2023-06-14--01-27-28_2365017561881232972.csv?dl=1'
urlsleep615 = 'https://www.dropbox.com/s/q37nqho2xmprdgz/mindMonitor_2023-06-15--00-27-29_8457076698113972093.csv?dl=1'
urlbreathingshorttest = 'https://www.dropbox.com/s/d5twz43yv7xqrwl/breathing_example_1035am.m4a?dl=1'
urlbreathingshorttestmuse = 'https://www.dropbox.com/s/ykv1p3ao8jt1a39/mindMonitor_2023-07-12--10-35-02_8991128292079486402_breathing_test.csv?dl=1'
urlaudiobreathsleep71623 = 'https://www.dropbox.com/s/nusz42jgkdv6n5x/sleep71623135.m4a?dl=1'
urlsleep719 = 'https://www.dropbox.com/s/dw9iqtfx9djpekc/mindMonitor_2023-07-19--01-28-39_673531789350893158_muses2.csv?dl=1'
urlsleep721 = 'https://www.dropbox.com/s/ce7gh2j95vrsh9p/mindMonitor_2023-07-21--08-32-56_7241060858940157562_morning_dreamt.csv?dl=1'
urlsleep724 = 'https://www.dropbox.com/s/yu41zvnbxb5uwn3/mindMonitor_2023-07-23--00-23-56_disconap_insomnia_sweets_processedsweets_2_maybe_reduction.csv?dl=1'









In [ ]:
#load audio files

import io
import requests

# Get the Dropbox URL of the object

# Download the object from Dropbox
response = requests.get(urlbreathingshorttest, stream=True)

# REWRITE FILE NAME Save the object in Google Colab's file structure
with open("breathing_example.m4a", "wb") as f:
  for chunk in response.iter_content(chunk_size=1024):
    f.write(chunk)

In [ ]:
#load file

dfraw = pd.read_csv(urlsleep615)


df = dfraw

In [ ]:
df.columns

In [ ]:
#pd.to_timedelta(df['TimeStamp']).head()
from datetime import datetime

df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
time0 = df['TimeStamp'].loc[0]

df['elapsed'] = 'NA'

#df['Results'] = ['Pass' if m>=33 else 'Fail' for m in df['Marks']]
for row in range(1,len(df),1):
    time1 = df['TimeStamp'].loc[row]
    time1 = pd.Timestamp(time1)
    df['elapsed'].loc[row] = pd.Timedelta(time1-time0).total_seconds()

df['elapsed'] = pd.to_numeric(df['elapsed'], errors='coerce')
df.head()



In [ ]:
#for original dataset

import numpy as np
# conditions = [
#     (df['elapsed'] <= 2),
#     (df['elapsed'] > 2) & (df['elapsed'] <= 9),
#     (df['elapsed'] > 9) & (df['elapsed'] <= 15),
#     (df['elapsed'] > 15)
#     ]

# # create a list of the values we want to assign for each condition
# values = ['tier_4', 'tier_3', 'tier_2', 'tier_1']

# # create a new column and use np.select to assign values to it using our lists as arguments
# df['tier'] = np.select(conditions, values)

# display updated DataFrame
# df.head()

conditions = [
    (df['elapsed'] <= 12),
    (df['elapsed'] > 12) & (df['elapsed'] <= 70),
    (df['elapsed'] > 77) & (df['elapsed'] <= 134),
     (df['elapsed'] > 134) & (df['elapsed'] <= 237),
     (df['elapsed'] > 237) & (df['elapsed'] <= 370),
    (df['elapsed'] > 370)
    ]

# create a list of the values we want to assign for each condition
values = ['eyesClosed1', 'thinking', 'eyesClosed2', 'intervention', 'postTreatment', 'finish']

# create a new column and use np.select to assign values to it using our lists as arguments
df['mindful'] = np.select(conditions, values)



eyesClosed1 = 12 #00:12
thinking = 70 #01:10
eyesClosed2 = 77 #01:17
intervention = 134 #02:14
postTreatment = 237 #03:57
finish = 370 #06:10

# display updated DataFrame
df.head(50)

In [ ]:
#for stimulant limted dataset

import numpy as np

conditions = [
    (df['elapsed'] <= 40),
    (df['elapsed'] > 40)
    ]

# create a list of the values we want to assign for each condition
values = ['before intervention', 'after intervention']

# create a new column and use np.select to assign values to it using our lists as arguments
df['mindful'] = np.select(conditions, values)

# display updated DataFrame
df.head(50)

In [ ]:
# dataMin = readData.min().min()
# dataMax = readData.max().max()
# normData = readData.copy()

# for col in normData.columns:
#     normData[col] = normData[col]-dataMin / dataMax-dataMin

df['Theta_AF7_norm'] = df['Theta_AF7']-df['Theta_AF7'].min() / (df['Theta_AF7'].max()-df['Theta_AF7'].min())
df['Beta_AF7_norm'] = df['Beta_AF7']-df['Beta_AF7'].min() / (df['Beta_AF7'].max()-df['Beta_AF7'].min())
df['ThetaBetaRatio_AF7_norm'] = df['Theta_AF7_norm'] / df['Beta_AF7_norm']



df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#grouped =df.groupby(['mindful'])['ThetaBetaRatio_AF7_norm'].median()
grouped =df.groupby(['mindful'])['Gamma_AF7'].median()

#for original dataset
print(grouped.reset_index(inplace=False))

# sns.lineplot(x='mindful', y='ThetaBetaRatio_AF7_norm', data=grouped.reset_index())

#for stimulant limited dataset

sns.histplot(x='mindful', y='Gamma_AF7', data=grouped.reset_index())


plt.xticks(rotation=45)

In [ ]:
print(df.index)
from scipy.interpolate import interp1d

#df['interpolated'] = interp1d(df['elapsed'], df['ThetaBetaRatio_AF7_norm'], kind='cubic')


#ax = sns.lineplot(x='elapsed', y='ThetaBetaRatio_AF7_norm', data=df.sample(frac=1))
ax = sns.lineplot(x='TimeStamp', y='Gamma_AF7', data=dfs)

#ax2 = sns.lineplot(x='elapsed', y='Gamma_AF7', data=df.sample(frac=0.2))

ax.set(ylim=(-0.7, 6))

In [ ]:
dfsec = df.rolling(256, min_periods=1).mean()
dfsec.head()
#dfs = dfsec.iloc[::30, :]

# exploratory analysis

In [ ]:
#load
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#url = 'https://www.dropbox.com/s/uq60t8qy33i21dt/mindMonitor_2023-02-14--09-43-07_huberman_cyclic_breath_work.csv?dl=1'
dfraw = pd.read_csv(urlsleep724)

dfraw.head()
df = dfraw

In [ ]:
# reduce file size

# df = (dfraw.select_dtypes('number')
#    .rolling(256, min_periods=1).mean()
#    .join(dfraw.select_dtypes(exclude='number'))
#    [dfraw.columns]
#  )

# df = df[::512]

# print ('length of df', len(df))

# df.head(20)

#reduce 0.5 second intervals to 10 second rolling average

df = (dfraw.select_dtypes('number')
   .rolling(150, min_periods=1).mean()
   .join(dfraw.select_dtypes(exclude='number'))
   [dfraw.columns]
 )

df.head(20)

In [ ]:
#norming data and ratio


df['Theta_AF8_norm'] = (df['Theta_AF8']-df['Theta_AF8'].min()) / ((df['Theta_AF8'].max()-df['Theta_AF8'].min()))
df['Theta_AF8_norm'] = (df['Theta_AF8']-df['Theta_AF8'].min()) / ((df['Theta_AF8'].max()-df['Theta_AF8'].min()))

df['Beta_AF8_norm'] = (df['Beta_AF8']-df['Beta_AF8'].min()) / ((df['Beta_AF8'].max()-df['Beta_AF8'].min()))
df['Beta_AF7_norm'] = (df['Beta_AF7']-df['Beta_AF7'].min()) / ((df['Beta_AF7'].max()-df['Beta_AF7'].min()))


df['Beta_AF8_af7_combined_norm'] = df['Beta_AF8_norm'] + df['Beta_AF7_norm']



df['ThetaBetaRatio_AF8_norm'] = df['Theta_AF8_norm'] / df['Beta_AF8_norm']

df['Delta_AF7_norm'] = (df['Delta_AF7']-df['Delta_AF7'].min()) / ((df['Delta_AF7'].max()-df['Delta_AF7'].min()))


df.head(20)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Convert time column to datetime
df['time'] = pd.to_datetime(df['TimeStamp'])

# Resample the data to minute intervals and calculate the mean
df_resampled = df.resample('1T', on='time').mean()

# Reset the index
df_resampled.reset_index(inplace=True)

# Calculate the relative frequency
total_power = np.power(10, df_resampled["Alpha_AF8"]) + np.power(10, df_resampled["Beta_AF8"]) + np.power(10, df_resampled["Theta_AF8"]) + np.power(10, df_resampled["Delta_AF8"])
df_resampled["Alpha_AF8"] = np.power(10, df_resampled["Alpha_AF8"]) / total_power
df_resampled["Beta_AF8"] = np.power(10, df_resampled["Beta_AF8"]) / total_power
df_resampled["Theta_AF8"] = np.power(10, df_resampled["Theta_AF8"]) / total_power
df_resampled["Delta_AF8"] = np.power(10, df_resampled["Delta_AF8"]) / total_power

# Melt the dataframe to long format for easier plotting with seaborn
df_melted = df_resampled.melt(id_vars='time', value_vars=['Alpha_AF8', 'Beta_AF8', 'Theta_AF8', 'Delta_AF8'])

# Create the line plot
plt.figure(figsize=(12,6))
sns.lineplot(data=df_melted, x='time', y='value', hue='variable')
plt.title("Relative Brain Wave Frequencies for AF8 Channel")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np


fig, ax = plt.subplots(figsize=(15, 8))

#ax = sns.lineplot(x='TimeStamp', y='Beta_AF8_af7_combined_norm', data=df, ax=ax )

ax = sns.lineplot(x='TimeStamp', y='Delta_AF7', data=df, ax=ax )

ax.set_xlabel('TimeStamp')
ax.set_ylabel('Normalized Values')
plt.show()

ax.set(ylim=(-0.5, 3))
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))

#ppg analysis

50hz is suggeste minimal sampling rate for heart reate variability

https://www.sciencedirect.com/science/article/pii/S1746809421001865

filters for ppg
https://www.nature.com/articles/sdata201876#:~:text=In%20addition%20to%20physiological%20information,chosen%20according%20to%20their%20purpose.




In [ ]:
len(df)

In [ ]:
dfs = df[::5]
print(len(dfs))

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

dfs = df.iloc[:30000:10]

# Create a figure with two subplots
fig, axes = plt.subplots(3, 1, figsize=(20, 5))

# Plot the first lineplot
sns.lineplot(data=dfs, x='TimeStamp', y="PPG_IR", ax=axes[0])
axes[0].set(xlabel='TimeStamp', ylabel='PPG_IR')

# Plot the second lineplot
sns.lineplot(data=dfs, x='TimeStamp', y="PPG_Red", ax=axes[1])
axes[1].set(ylabel='PPG_Red')

# Plot the second lineplot
sns.lineplot(data=dfs, x='TimeStamp', y="PPG_Ambient", ax=axes[2])
axes[1].set(ylabel='PPG_Red')

# Show the figure
plt.show()


##entropy

In [ ]:
!pip install antropy
import antropy as ant

In [ ]:
len(df)

#https://www.analyticsvidhya.com/blog/2020/11/entropy-a-key-concept-for-all-data-science-beginners/

In [ ]:
dfs = df.iloc[3500:]
x = dfs['Beta_AF8_norm'][df['Beta_AF8_norm'].notna()]
# Permutation entropy
print(ant.perm_entropy(x, normalize=True))
# Spectral entropy
print(ant.spectral_entropy(x, sf=100, method='welch', normalize=True))
# Singular value decomposition entropy
#print(ant.svd_entropy(x, normalize=True))
# Approximate entropy
#print(ant.app_entropy(x))
# Sample entropy
#print('sample entroy', ant.sample_entropy(x))
# Hjorth mobility and complexity
print(ant.hjorth_params(x))
# Number of zero-crossings
print(ant.num_zerocross(x))
# Lempel-Ziv complexity
print(ant.lziv_complexity('01111000011001', normalize=True))

#sleep analysis

In [ ]:
import yasa
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load EEG data
f = np.load('/content/data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data, ch_names = f['data'], f['chan']
sf = 100.

# Load hypnogram
hypno_30s = np.loadtxt('data_full_6hrs_100Hz_hypno_30s.txt')
hypno = yasa.hypno_upsample_to_data(hypno=hypno_30s, sf_hypno=(1/30), data=data, sf_data=sf)
print(hypno.shape, 'Unique values =', np.unique(hypno))

In [ ]:
#contruct yasa and mne array

# sampling_freq = 200  # in Hertz
# n_channels = 32
# info = mne.create_info(n_channels, sfreq=sampling_freq)
# print(info)

# data = np.array([sine, cosine])

# info = mne.create_info(ch_names=['10 Hz sine', '5 Hz cosine'],
#                        ch_types=['misc'] * 2,
#                        sfreq=sampling_freq)


n_channels = 1
sampling_freq = 2  # in Hertz
info = mne.create_info(n_channels, sfreq=sampling_freq)
print(info)


data = np.array(df['RAW_AF8'].to_list())

data = np.atleast_2d(data)

info = mne.create_info(ch_names=['rawf8'],
                        ch_types=['eeg'],
                        sfreq=sampling_freq)

simulated_raw = mne.io.RawArray(data, info)

In [ ]:
print(simulated_raw._data[0,0:200])

In [ ]:
df.columns
#'RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10'



## filtering raw data


In [ ]:
#!pip install mne
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import mne
#urlsleep = 'https://www.dropbox.com/s/hu50afgorep21c6/mindMonitor_2023-04-03--01-28-07_sleep.csv?dl=1'
#dfraw = pd.read_csv('/content/mindMonitor_2023-01-18--14-37-14.csv')

#dfraw = pd.read_csv('/content/mindMonitor_2023-02-15--14-48-03_stiumant_limited_interventions.csv')
#dfraw = pd.read_csv('/content/mindMonitor_2023-02-23--08-01-50_eyesopen_eyesclose_mantrameditation_eyesopen_eyesclosed.csv')
#dfraw = pd.read_csv('/content/mindMonitor_2023-02-23--08-01-50_eyesopen_eyesclose_mantrameditation_eyesopen_eyesclosed.csv')
#urlsleep58 = 'https://www.dropbox.com/s/9r516639sms98sj/mindMonitor_2023-05-07--23-31-37_5188390995226691787_sleep.csv?dl=1'
dfraw = pd.read_csv(urlsleep615)


df = dfraw

In [ ]:
print(df.TimeStamp.min())
print(df.TimeStamp.max())

In [ ]:
import pandas as pd
import mne
import numpy as np

# Get the RAW_AF8 column
raw_af8 = df['RAW_AF8'].values

# Convert the data to a floating-point type
raw_af8 = raw_af8.astype(float)

# Interpolate missing values
raw_af8_interp = np.interp(np.arange(len(raw_af8)), np.where(~np.isnan(raw_af8))[0], raw_af8[~np.isnan(raw_af8)])

# Define the filter parameters
order = 3  # Filter order
ftype = 'butter'  # Filter type

# Specify the cutoff frequency
f_p = 40.0  # Upper cutoff frequency (Hz)
f_s = 1.0  # Lower cutoff frequency (Hz)

# Set the sampling frequency
sfreq = 256.0  # Sampling frequency (Hz)

# Create the filter
iir_params = dict(order=order, ftype=ftype)
filt = mne.filter.create_filter(raw_af8_interp, sfreq, l_freq=f_s, h_freq=f_p, method="iir", iir_params=iir_params, verbose=True)

# Apply the filter to the data
filtered_af8 = mne.filter.filter_data(raw_af8_interp, sfreq, l_freq=f_s, h_freq=f_p, method="iir", iir_params=iir_params)

# Check the filtered data range
print("Filtered Data Range:")
print("Minimum value:", min(filtered_af8))
print("Maximum value:", max(filtered_af8))

df['filtered_AF8'] = filtered_af8

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(30, 3))
sns.lineplot(data=df.iloc[260800:268480], x='TimeStamp', y='filtered_AF8')
plt.show()

In [ ]:
# Get the time stamp column
time_stamps = np.array(df['TimeStamp'])

# Get the RAW_AF8 column
raw_af8 = df['RAW_AF8'].values
raw_af7 = df['RAW_AF7'].values

# Convert the data to a floating-point type
raw_af8 = raw_af8.astype(np.float64)
raw_af7 = raw_af7.astype(np.float64)

# Ensure the conversion has been successful
assert raw_af8.dtype == np.float64, f"Unexpected data type: {raw_af8.dtype}"
assert raw_af7.dtype == np.float64, f"Unexpected data type: {raw_af7.dtype}"

print(raw_af8.dtype)
print(raw_af7.dtype)

# Create a high-pass filter at 2 Hz and a low-pass filter at 45 Hz
l_freq = 3.5
h_freq = 8.5
sfreq = 256.0  # explicitly defined as float

# Apply the filter to the data
filtered_af8 = mne.filter.filter_data(raw_af8, sfreq, l_freq=l_freq, h_freq=h_freq, fir_design='firwin')
filtered_af7 = mne.filter.filter_data(raw_af7, sfreq, l_freq=l_freq, h_freq=h_freq, fir_design='firwin')

# Create a new column in the DataFrame for filtered values
df['filtered_AF8'] = filtered_af8
df['filtered_AF7'] = filtered_af7

# Calculate the average of filtered values and create a new column "frontfil"
df['frontfil'] = (df['filtered_AF8'] + df['filtered_AF7']) / 2


In [ ]:
# Get the time stamp column
time_stamps = np.array(df['TimeStamp'])

# Get the RAW_AF8 and RAW_AF7 columns
raw_af8 = df['RAW_AF8'].values
raw_af7 = df['RAW_AF7'].values

# Convert the data to a floating-point type
raw_af8 = raw_af8.astype(np.float64)
raw_af7 = raw_af7.astype(np.float64)

# Ensure the conversion has been successful
assert raw_af8.dtype == np.float64, f"Unexpected data type: {raw_af8.dtype}"
assert raw_af7.dtype == np.float64, f"Unexpected data type: {raw_af7.dtype}"

print(raw_af8.dtype)
print(raw_af7.dtype)

# Sampling frequency
sfreq = 256.0  # explicitly defined as float

# Slow wave sleep frequency range
l_freq_slow = 0
h_freq_slow = 4.5

# N2 sleep frequency range
l_freq_n2 = 4.6
h_freq_n2 = 8.6

# Apply the slow wave sleep filter to the data
filtered_af8_slow = mne.filter.filter_data(raw_af8, sfreq, l_freq=l_freq_slow, h_freq=h_freq_slow, fir_design='firwin')
filtered_af7_slow = mne.filter.filter_data(raw_af7, sfreq, l_freq=l_freq_slow, h_freq=h_freq_slow, fir_design='firwin')

# Apply the N2 sleep filter to the data
filtered_af8_n2 = mne.filter.filter_data(raw_af8, sfreq, l_freq=l_freq_n2, h_freq=h_freq_n2, fir_design='firwin')
filtered_af7_n2 = mne.filter.filter_data(raw_af7, sfreq, l_freq=l_freq_n2, h_freq=h_freq_n2, fir_design='firwin')

# Create new columns in the DataFrame for filtered values
df['filtered_AF8_slow'] = filtered_af8_slow
df['filtered_AF7_slow'] = filtered_af7_slow
df['filtered_AF8_n2'] = filtered_af8_n2
df['filtered_AF7_n2'] = filtered_af7_n2

# Calculate the average of filtered values for each band and create new columns
df['frontfil_slow'] = (df['filtered_AF8_slow'] + df['filtered_AF7_slow']) / 2
df['frontfil_n2'] = (df['filtered_AF8_n2'] + df['filtered_AF7_n2']) / 2


## graph large with datashader

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
from IPython.display import display

# Total number of rows to plot
total_rows = 460800

# Rows per plot
chunk_size = 7680

# Number of chunks to plot
n_chunks = total_rows // chunk_size

for i in range(n_chunks):
    start_row = i * chunk_size
    end_row = start_row + chunk_size - 1  # Subtract 1 since end_row is inclusive

    # Get the timestamp for the start and end rows
    start_time = df.iloc[start_row]['TimeStamp']
    end_time = df.iloc[end_row]['TimeStamp']
    print(f"Plotting from {start_time} to {end_time}")

    # Slice the DataFrame to include only the current chunk of rows
    df_chunk = df.iloc[start_row:end_row+1].copy()  # Create a copy to avoid modifying original data

    # Convert datetime index to numerical representation (seconds since 1970-01-01)
    df_chunk['time_numeric'] = df_chunk.index.astype(int) / 10**9

    # Create the datashader canvas
    canvas = ds.Canvas(plot_width=1500, plot_height=300)

    # Aggregate the data (binning into pixels)
    agg = canvas.line(df_chunk, 'time_numeric', 'filtered_AF8')

    # Transform the aggregated data into an image
    img = tf.shade(agg)

    # Display the image
    display(img)


In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
from colorcet import fire, bgy

# Total number of rows to plot
total_rows = 460800

# Rows per plot
chunk_size = 7680

# Number of chunks to plot
n_chunks = total_rows // chunk_size


for i in range(n_chunks):
    start_row = i * chunk_size
    end_row = start_row + chunk_size
    print(f"Plotting rows {start_row} to {end_row}")

    # Slice the DataFrame to include only the current chunk of rows
    df_chunk = df.iloc[start_row:end_row].copy()  # Create a copy to avoid modifying original data

    # Convert datetime index to numerical representation (seconds since 1970-01-01)
    df_chunk['TimeStamp'] = pd.to_datetime(df_chunk['TimeStamp'])

    df_chunk['time_numeric'] = df_chunk['TimeStamp'].astype(int) / 10**9

    # Create the datashader canvas
    canvas = ds.Canvas(plot_width=1500, plot_height=300)

    # Aggregate the data (binning into pixels)
    agg_slow = canvas.line(df_chunk, 'time_numeric', 'frontfil_slow')
    agg_n2 = canvas.line(df_chunk, 'time_numeric', 'frontfil_n2')

    # Transform the aggregated data into images
    img_slow = tf.shade(agg_slow, cmap=fire)
    img_n2 = tf.shade(agg_n2, cmap=bgy)

    # Overlay the two images
    img_overlay = tf.stack(img_slow, img_n2)

    # Display the image
    display(img_overlay)


## sleep 614

sleep 614 deep sleep at around 10:04 pm and REM at around 1am


In [ ]:
dfraw = pd.read_csv(urlsleep614)

df = dfraw


In [ ]:
df.head()
print (df.TimeStamp.max())
print (df.TimeStamp.min())

In [ ]:
df['time'] = pd.to_datetime(df['TimeStamp'])



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create new DataFrame with 'time' as the index
dft = df.set_index('time')

# Slice the DataFrame between 6:30 AM and 7:00 AM
dft_sliced = dft.between_time('02:30', '03:00')

dft_30s = dft_sliced.iloc[:7680].reset_index()


plt.figure(figsize=(12,6))
sns.lineplot(x='time', y='RAW_AF8', data=dft_30s) #first 30 seconds
plt.title('RAW_AF8 Line Plot for the First 30 Seconds')
plt.show()



In [ ]:
#attempt filtered

# Get the RAW_AF8 column
raw_af8 = dft_30s['RAW_AF8'].values

# Convert the data to a floating-point type
raw_af8 = raw_af8.astype(np.float64)

# Ensure the conversion has been successful
assert raw_af8.dtype == np.float64, f"Unexpected data type: {raw_af8.dtype}"

print(raw_af8.dtype)

# Create a high-pass filter at 0.1 Hz
sfreq = 256.0  # explicitly defined as float

# Apply the filter to the data
filtered_data = mne.filter.filter_data(raw_af8, sfreq, l_freq=0.5, h_freq=None, fir_design='firwin')

# Create a new column in the DataFrame
dft_30s['f8f'] = filtered_data

plt.figure(figsize=(12,6))
sns.lineplot(x='time', y='f8f', data=dft_30s) #first 30 seconds
plt.title('filtered RAW_AF8 Line Plot for the First 30 Seconds')
plt.show()

In [ ]:
dft_30s.head()

##large sleep section imported from git

for now, load sleep files manually. not working from url

In [ ]:
#!pip install mne
#!pip install yasa
import pandas as pd
import mne
import numpy as np
import yasa

#urlsleep58 = 'https://www.dropbox.com/s/9r516639sms98sj/mindMonitor_2023-05-07--23-31-37_5188390995226691787_sleep.csv?dl=1'
#urlnap58 = 'https://www.dropbox.com/s/fbjsqhjlhuocps4/mindMonitor_2023-05-08--14-47-48_7822430682462145648_nap.csv?dl=1'
#dfraw = pd.read_csv(urlnap58, skiprows=[169541, 202609])
#dfs = dfraw[['RAW_AF8']]

In [ ]:
# import urllib.request
# from tqdm import tqdm

# urlnap58 = 'https://www.dropbox.com/s/fbjsqhjlhuocps4/mindMonitor_2023-05-08--14-47-48_7822430682462145648_nap.csv?dl=1'
# # Download the file with a progress bar

# with urllib.request.urlopen(urlnap58) as response:
#     sleepfile = response.read().decode("utf-8")

# # Save the file to Colab memory
# with open("sleepfile.csv", "w") as f:
#     f.write(sleepfile)

In [ ]:
#manually loaded
df = pd.read_csv('/content/mindMonitor_2023-05-08--14-47-48_7822430682462145648_nap (2).csv', error_bad_lines=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head(300)

In [ ]:
df["time"] = pd.to_datetime(df["TimeStamp"])


In [ ]:
df['RAW_AF8'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt



# Get the time periods
#start_time = 2023-05-08 14:47:48.254
#end_time = 2023-05-08 14:49:48.254

# Filter the dataframe to the specified time periods
#df = df[(df['timestamp'] > start_time) & (df['timestamp'] < end_time)]

# Create a line plot of the 'RAW_AF8' column, with the x-axis in seconds
#sns.lineplot(x=df['time'].dt.second, y='RAW_AF8', data=df)
fig, ax = plt.subplots()

sns.lineplot(x=df['time'], y='RAW_AF7', data=df.sample(100))
ax.set_ylim(750, 850)



# Show the plot
plt.show()


In [ ]:
dfs = dfraw[['RAW_AF8']]

def df_to_raw(df):
    raw = np.array(df)
    return raw

# changes: Set 'sfreq' to the minimum 81
# When it comes to electrophysiological data, MNE works on the basis of samples, not time points
# The duration of the recording can then be calculated by dividing the number of samples by the sampling frequency
# Using data from urlspeep: samples/frequency -> 35720/256 = 139.531 seconds
# OpenAI Prompt:
#   # The duration of recording of a mne object
def raw_to_mne(raw):
    info = mne.create_info(ch_names=['RAW_AF8'],
                           sfreq=256,
                           ch_types=['eeg'])
    mne_raw = mne.io.RawArray(raw, info)
    return mne_raw

# Turns the MNE object into the YASA package
def mne_to_yasa(mne_raw):
    yasa_raw = yasa.SleepStaging(mne_raw, eeg_name='RAW_AF8')
    return yasa_raw

# Initialize the sleep staging instance
sls = mne_to_yasa(raw_to_mne(df_to_raw(dfs.values.T)))

# Get the predicted sleep stages
hypno = sls.predict()
# Get the predicted probabilities
proba = sls.predict_proba()
# Get the confidence
confidence = proba.max(axis=1)
# Plot the predicted probabilities
sls.plot_predict_proba()

#Audio and breath analysis

In [ ]:
import io
import requests

def download_file_from_dropbox(url, filename):
    response = requests.get(url, stream=True)
    with io.open(filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)

url = urlaudiobreathsleep71623

filename = "file.mp3"
download_file_from_dropbox(url, filename)



In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
#audio_path = '/content/breathing_example.m4a'
audio, sr = librosa.load('/content/file.mp3')

# Compute the spectrogram
spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)

# Convert to dB scale
spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

# Plot the spectrogram with the suggested color palette
plt.figure(figsize=(10, 4))
librosa.display.specshow(spectrogram_db, sr=sr, x_axis='time', y_axis='mel', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.tight_layout()
plt.show()


In [ ]:
#load muse file for short breathing test
dfraw = pd.read_csv(urlbreathingshorttestmuse)
df = dfraw

In [ ]:
df.head()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from datetime import datetime, timedelta
from scipy.signal import butter, lfilter

# Load the entire audio file
audio_path = '/content/breathing_example.m4a'
audio_full, sr = librosa.load(audio_path)

# Compute the spectrogram
spectrogram = librosa.feature.melspectrogram(y=audio_full, sr=sr)

# Convert to dB scale
spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

#new dataframe
dfs = df.dropna(subset=['PPG_IR'])

# Convert 'TimeStamp' column to datetime
dfs['TimeStamp'] = pd.to_datetime(df['TimeStamp'])

#frequency
fs = 256.0  # Sampling frequency
lowcut = 0.4  # Low cutoff frequency
highcut = 4.0  # High cutoff frequency
nyq = 0.5 * fs  # Nyquist frequency
low = lowcut / nyq
high = highcut / nyq
b, a = butter(3, [low, high], btype='band')

# Apply Butterworth filter
dfs['filtered_PPG_IR'] = lfilter(b, a, dfs['PPG_IR'].values)

# Create two subplots: one for the unfiltered PPG data, one for the filtered PPG data
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

# Plot the unfiltered PPG data
sns.lineplot(data=dfs[::2], x='TimeStamp', y="PPG_IR", ax=ax[0])
ax[0].set(xlabel='TimeStamp', ylabel='Unfiltered PPG_IR')

# Plot the filtered PPG data
sns.lineplot(data=dfs[::2], x='TimeStamp', y="filtered_PPG_IR", ax=ax[1])
ax[1].set(xlabel='TimeStamp', ylabel='Filtered PPG_IR')

plt.tight_layout()
plt.show()

import IPython.display as ipd
ipd.Audio(audio_path, rate=sr)


In [ ]:
print(dfs['PPG_IR'].isna().sum())
print (len(dfs['PPG_IR']))

dfs = dfs.dropna(subset=['PPG_IR'])


# Butterworth bandpass filter design
fs = 256.0  # Sampling frequency
lowcut = 0.5  # Low cutoff frequency
highcut = 3.0  # High cutoff frequency
nyq = 0.5 * fs  # Nyquist frequency
low = lowcut / nyq
high = highcut / nyq
b, a = butter(2, [low, high], btype='band')

# Apply the filter
dfs['PPG_IR_Filtered'] = filtfilt(b, a, dfs['PPG_IR'])

dfs['PPG_IR_Filtered'].describe()



#ECG data

In [ ]:
import pandas as pd

# Read the txt file
with open("/content/2023-04-12 11-56-41.txt", "r") as f:
  lines = f.readlines()

# Remove the \n character from each line
lines = [line.strip("\n") for line in lines]

# Convert the lines to integers
lines = [int(line) for line in lines]

# Create a Pandas DataFrame
dfg = pd.DataFrame(lines, columns=["data"])

# Print the DataFrame
print(dfg)


In [ ]:
sns.lineplot(data=dfg[0:1000], x=dfg.index[:1000], y='data')


In [ ]:
#status

add time stamps to polar EMC

import datetime
import time

# Get the name of the text file
filename = "2023-04-12 11-56-41"

# Get the timestamp
timestamp = datetime.datetime.strptime(filename, "%Y-%m-%d %H-%M-%S")

# Open the text file
with open(filename, "r") as f:
  lines = f.readlines()

# Add the timestamp to the lines
for i, line in enumerate(lines):
  new_line = "%s, %.3f" % (timestamp + timedelta(seconds=i / 130), line)
  lines[i] = new_line

# Save the text file
with open(filename, "w") as f:
  f.writelines(lines)


#upwork notebook

In [ ]:
import matplotlib.pyplot as plt
import mne
import numpy as np
import os
import pandas as pd
from openpyxl import load_workbook
import ipywidgets as widgets
from matplotlib.backends.backend_pdf import PdfPages

#function

different_frequency_bands = 0
def plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, fig_title):
    fig, ax = plt.subplots(figsize=(20, 10))
    ylimit = 50
    shift_y = 500
    for i in range(data_epoch.shape[0]):
        shift = i * shift_y
        ax.plot(time_epoch, data_epoch[i, :] + shift)
        (_, caps, _) = ax.errorbar(x=time_epoch[0], y=shift,
                                   yerr=ylimit, color='k', capsize=5)
        for cap in caps:
            cap.set_markeredgewidth(1)
        # *2 because it goes from -ylimit to +ylimit
        ax.text(x=time_epoch[0]-0.5, y=shift,
                s=str(ylimit*2)+'µV', ha='center')

    y_ticks = np.arange(data_epoch.shape[0]) * shift_y
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(ch_names)

    ax.set_xlabel('Time (sec)')
    ax.set_title('Epoch number:' + str(epoch+1) + ', ' + fig_title)
    pdf.savefig(fig)

def handle_text_submit(sender):
    epoch = int(sender.value)
    print('-'*300)
    print('Epoch number:', epoch)
    epoch_length = (epoch-1)*30*sfreq
    epoch_samples = [epoch_length, (epoch_length) +
                    sfreq*30]  # 30-second epoch
    if (epoch_samples[1] > hold_data.shape[1]):
        raise Exception(
            'This epoch exceeds the length of the available data, please try again with a smaller value')

    data_epoch = hold_data[:, epoch_samples[0]:epoch_samples[1]]
    time_epoch = raw.times[epoch_samples[0]:epoch_samples[1]]
    plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Broad')

    if different_frequency_bands==1:
      # looking at specific frequency bands
      data_epoch = raw_delta.get_data()[:, epoch_samples[0]:epoch_samples[1]]
      plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Delta')

      data_epoch = raw_theta.get_data()[:, epoch_samples[0]:epoch_samples[1]]
      plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Theta')

      data_epoch = raw_alpha.get_data()[:, epoch_samples[0]:epoch_samples[1]]
      plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Alpha')

      data_epoch = raw_sigma.get_data()[:, epoch_samples[0]:epoch_samples[1]]
      plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Sigma')
    plt.show()
    sender.value = ''

##load data
#loc = 'https://www.dropbox.com/s/q37nqho2xmprdgz/mindMonitor_2023-06-15--00-27-29_8457076698113972093.csv?dl=1'
loc = urlsleep721
dfraw = pd.read_csv(loc)
df = dfraw


raw_data = df.loc[1:, ['RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10']]

# dropping missing values
raw_data = raw_data.dropna(
    subset=['RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10'])

raw_data = raw_data[:].values


##preprocessing

sfreq = 256
ch_types = ["eeg"]*raw_data.shape[1]
ch_names = ["TP9", "AF7", "AF8", "TP10"]
montage = mne.channels.make_standard_montage("standard_1020")
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
samples = raw_data.T

# checking that there are no missing values
np.sum((np.isnan((samples[0, :]))) + 0)

# demean
# samples = samples - np.mean(samples, axis=1)[:, np.newaxis]
# common average reference
# get the mean across channels and then we repeat this to match the data and then subtract it from the data
# samples = samples - \
#     np.repeat(np.mean(samples, axis=0)[
#               np.newaxis, :], samples.shape[0], axis=0)
# TP9 and TP10 as reference
# samples = samples - \
#     np.repeat(np.mean(np.vstack((samples[0, :], samples[3, :])), axis=0)[
#               np.newaxis, :], samples.shape[0], axis=0)

raw = mne.io.RawArray(samples, info)


# raw = raw.copy().notch_filter(freqs=[50])
# raw = raw.copy().resample(100, npad='auto')
# raw = raw.copy().filter(l_freq=0.5, h_freq=15)
# if different_frequency_bands==1:
#   raw_delta = raw.copy().filter(l_freq=0.1, h_freq=4)
#   raw_theta = raw.copy().filter(l_freq=4, h_freq=8)
#   raw_alpha = raw.copy().filter(l_freq=8, h_freq=12)
#   raw_sigma = raw.copy().filter(l_freq=11, h_freq=16)  # for spindles
#sfreq = 100

##preprocess with butterworth edited

sfreq = 256
ch_types = ["eeg"]*raw_data.shape[1]
ch_names = ["TP9", "AF7", "AF8", "TP10"]
montage = mne.channels.make_standard_montage("standard_1020")
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
samples = raw_data.T

# checking that there are no missing values
np.sum((np.isnan((samples[0, :]))) + 0)

# demean
# samples = samples - np.mean(samples, axis=1)[:, np.newaxis]
# common average reference
# get the mean across channels and then we repeat this to match the data and then subtract it from the data
# samples = samples - \
#     np.repeat(np.mean(samples, axis=0)[
#               np.newaxis, :], samples.shape[0], axis=0)
# TP9 and TP10 as reference
# samples = samples - \
#     np.repeat(np.mean(np.vstack((samples[0, :], samples[3, :])), axis=0)[
#               np.newaxis, :], samples.shape[0], axis=0)

raw = mne.io.RawArray(samples, info)

# applying the bandpass Butterworth filter
raw = raw.copy().filter(l_freq=0.5, h_freq=40, method='iir', iir_params={"order": 4, "ftype": "butter"})
#raw = raw.copy().resample(100, npad='auto')

# if different_frequency_bands==1:
#   raw_delta = raw.copy().filter(l_freq=0.1, h_freq=4)
#   raw_theta = raw.copy().filter(l_freq=4, h_freq=8)
#   raw_alpha = raw.copy().filter(l_freq=8, h_freq=12)
#   raw_sigma = raw.copy().filter(l_freq=11, h_freq=16)  # for spindles
#sfreq = 100

##display continuous data

hold_data = (raw.get_data())

fig = plt.figure(figsize=(20, 15))
plt.plot(raw.times[:510*30*256], hold_data[:, :510*30*256].T)
plt.xlabel("Time (sec)")
plt.ylabel("Amplitude (uV)")
plt.title("raw data")
plt.show()

##call functions and display

print('max epoch is: ' + str(int(np.floor((hold_data.shape[1]/(30*sfreq))))))
# textbox = widgets.Text(placeholder='Please enter the epoch number')
# textbox.on_submit(handle_text_submit)
pdf = PdfPages('Output figures.pdf')

for epoch in range(int(np.floor((hold_data.shape[1]/(30*sfreq)))) ):
  print('Epoch number:', epoch+1)
  epoch_length = (epoch)*30*sfreq
  epoch_samples = [epoch_length, (epoch_length) +
                  sfreq*30]  # 30-second epoch

  data_epoch = hold_data[:, epoch_samples[0]:epoch_samples[1]]
  time_epoch = raw.times[epoch_samples[0]:epoch_samples[1]]
  plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Broad')

  if different_frequency_bands==1:
    # looking at specific frequency bands
    data_epoch = raw_delta.get_data()[:, epoch_samples[0]:epoch_samples[1]]
    plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Delta')

    data_epoch = raw_theta.get_data()[:, epoch_samples[0]:epoch_samples[1]]
    plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Theta')

    data_epoch = raw_alpha.get_data()[:, epoch_samples[0]:epoch_samples[1]]
    plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Alpha')

    data_epoch = raw_sigma.get_data()[:, epoch_samples[0]:epoch_samples[1]]
    plot_sleep_EEG(epoch, ch_names, data_epoch, time_epoch, 'Sigma')
  plt.show()

pdf.close()
# display(textbox)


In [ ]:
##preprocess with butterworth edited

sfreq = 256
ch_types = ["eeg"]*raw_data.shape[1]
ch_names = ["TP9", "AF7", "AF8", "TP10"]
montage = mne.channels.make_standard_montage("standard_1020")
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
samples = raw_data.T

# checking that there are no missing values
np.sum((np.isnan((samples[0, :]))) + 0)

# demean
# samples = samples - np.mean(samples, axis=1)[:, np.newaxis]
# common average reference
# get the mean across channels and then we repeat this to match the data and then subtract it from the data
# samples = samples - \
#     np.repeat(np.mean(samples, axis=0)[
#               np.newaxis, :], samples.shape[0], axis=0)
# TP9 and TP10 as reference
# samples = samples - \
#     np.repeat(np.mean(np.vstack((samples[0, :], samples[3, :])), axis=0)[
#               np.newaxis, :], samples.shape[0], axis=0)

raw = mne.io.RawArray(samples, info)

# applying the bandpass Butterworth filter
raw = raw.copy().filter(l_freq=0.1, h_freq=50, method='iir', iir_params={"order": 3, "ftype": "butter"})
#raw = raw.copy().resample(100, npad='auto')

if different_frequency_bands==1:
  raw_delta = raw.copy().filter(l_freq=0.1, h_freq=4)
  raw_theta = raw.copy().filter(l_freq=4, h_freq=8)
  raw_alpha = raw.copy().filter(l_freq=8, h_freq=12)
  raw_sigma = raw.copy().filter(l_freq=11, h_freq=16)  # for spindles
#sfreq = 100

##display continuous data

hold_data = (raw.get_data())

fig = plt.figure(figsize=(20, 15))
plt.plot(raw.times[:510*30*256], hold_data[:, :510*30*256].T)
plt.xlabel("Time (sec)")
plt.ylabel("Amplitude (uV)")
plt.title("raw data")
plt.show()



#Mock data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#url = 'https://www.dropbox.com/s/z09umrre7qgg4ld/mindMonitor_2023-02-23--08-01-50_eyesopen_eyesclose_mantrameditation_eyesopen_eyesclosed.csv?dl=1'
url = 'https://www.dropbox.com/s/a5uwkqaq71mkrxp/mindMonitor_2023-02-17--18-29-44_choc_intervention.csv?dl=1'
#url = 'https://www.dropbox.com/s/pdin2ayuuybcpi0/mindMonitor_2023-02-22--08-57-17_mantra_meditation.csv?dl=1'
#url = 'https://www.dropbox.com/s/uq60t8qy33i21dt/mindMonitor_2023-02-14--09-43-07_huberman_cyclic_breath_work.csv?dl=1'
dfraw = pd.read_csv(url)

dfraw.head()

#df = dfraw[::256]

In [ ]:
dictionary = {'TimeStamp': {0: '2023-02-23 08:01:50.701',
  1: '2023-02-23 08:01:50.798',
  2: '2023-02-23 08:01:50.798',
  3: '2023-02-23 08:01:50.800',
  4: '2023-02-23 08:01:50.800'},
 'Delta_TP9': {0: np.nan,
  1: 0.8932789112449511,
  2: 0.8932789112449511,
  3: 0.8932789112449511,
  4: 0.8932789112449511},
 'Delta_AF7': {0: np.nan,
  1: -0.062321571240896,
  2: -0.0734485722420289,
  3: -0.0734485722420289,
  4: -0.0734485722420289}}

dfm = pd.DataFrame.from_dict(dictionary)

In [ ]:
#contruct yasa and mne array

# sampling_freq = 200  # in Hertz
# n_channels = 32
# info = mne.create_info(n_channels, sfreq=sampling_freq)
# print(info)

# data = np.array([sine, cosine])

# info = mne.create_info(ch_names=['10 Hz sine', '5 Hz cosine'],
#                        ch_types=['misc'] * 2,
#                        sfreq=sampling_freq)


n_channels = 1
sampling_freq = 256  # in Hertz
info = mne.create_info(n_channels, sfreq=sampling_freq)
print(info)


data = np.array(df['Delta_TP9'].to_list())

data = np.atleast_2d(data)

info = mne.create_info(ch_names=['rawf8'],
                        ch_types=['eeg'],
                        sfreq=sampling_freq)

simulated_raw = mne.io.RawArray(data, info)

In [ ]:
## reduce file size

df = (dfraw.select_dtypes('number')
   .rolling(500, min_periods=1).mean()
   .join(dfraw.select_dtypes(exclude='number'))
   [dfraw.columns]
 )

df = df[::256]

print (len(df))

In [ ]:

df = (dfraw.select_dtypes('number')
   .rolling(500, min_periods=1).mean()
   .join(dfraw.select_dtypes(exclude='number'))
   [dfraw.columns]
 )

df = df[::500]

print (len(df))

In [ ]:
#norming data and ratio

df['Theta_AF8_norm'] = (df['Theta_AF8']-df['Theta_AF8'].min()) / ((df['Theta_AF8'].max()-df['Theta_AF8'].min()))
df['Beta_AF8_norm'] = (df['Beta_AF8']-df['Beta_AF8'].min()) / ((df['Beta_AF8'].max()-df['Beta_AF8'].min()))
df['ThetaBetaRatio_AF8_norm'] = df['Theta_AF8_norm'] / df['Beta_AF8_norm']

df.head(20)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn.apionly as sns
import numpy as np


fig, ax = plt.subplots(figsize=(15, 8))

ax = sns.lineplot(x='TimeStamp', y='ThetaBetaRatio_AF8_norm', data=df, ax=ax )

#ax2 = sns.lineplot(x='TimeStamp', y='Gamma_AF8', data=dfraw, ax=ax)

ax.set(ylim=(-1, 3))
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_locator(ticker.MultipleLocator(3000))


In [ ]:
df = dfraw.ilocp

In [ ]:
x = dfraw.head(5)

x.iloc[:,0:3].to_dict()

In [ ]:
dictionary = {'TimeStamp': {0: '2023-02-23 08:01:50.701',
  1: '2023-02-23 08:01:50.798',
  2: '2023-02-23 08:01:50.798',
  3: '2023-02-23 08:01:50.800',
  4: '2023-02-23 08:01:50.800'},
 'Delta_TP9': {0: np.nan,
  1: 0.8932789112449511,
  2: 0.8932789112449511,
  3: 0.8932789112449511,
  4: 0.8932789112449511},
 'Delta_AF7': {0: np.nan,
  1: -0.062321571240896,
  2: -0.0734485722420289,
  3: -0.0734485722420289,
  4: -0.0734485722420289}}

dfm = pd.DataFrame.from_dict(dictionary)

#dfm.rolling(3).mean()

df.head()

In [ ]:
(df.select_dtypes('number')
   .rolling(3).mean()
   .join(df.select_dtypes(exclude='number'))
   [df.columns]
 )

In [ ]:
df.select_dtypes(exclude='number')[df.columns]

In [ ]:
dfsec = df.rolling(256, min_periods=1).mean()
dfsec.head()
#dfs = dfsec.iloc[::30, :]

df.index

In [ ]:
df_time = pd.DataFrame({'B': [0, 1, 2, np.nan, 4]},
                       index = [pd.Timestamp('20130101 09:00:00'),
                                pd.Timestamp('20130101 09:00:02'),
                                pd.Timestamp('20130101 09:00:03'),
                                pd.Timestamp('20130101 09:00:05'),
                                pd.Timestamp('20130101 09:00:06')])


df_time.rolling(2, min_periods=1).mean()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [0.301, 0.477, 0.602, 0.699, 0.778, 0.845, 0.903, 0.954, 1.000, 1.044]

df = pd.DataFrame({"X" : x, "Log2" : y})

df.plot(x="X", y="Log2")
plt.title("Log2 graph")
plt.show()

### live streaming